In [16]:
import json
import pandas as pd
import os
import numpy as np
pd.set_option("display.max_columns", None)

In [17]:
os.chdir("/Users/sid98/OMSA/04_CSE_6242_DVA/ProjectFiles/FDA")

In [18]:
with open("./fda.json", "r") as f:
    fda = json.load(f)
fda_df = pd.DataFrame(fda["results"])

PRODUCTS = []
keys = list(fda_df["products"][1][0].keys())
for i in range(len(fda_df)):
    products = fda_df["products"][i]
    appl_no = fda_df["application_number"][i]
    if str(products) != "nan":
        for item in products:
            item["applnNo"] = appl_no
            PRODUCTS.append(item)

    else:
        temp = {"applnNo": appl_no}
        for k in keys:
            temp[k] = np.nan
        PRODUCTS.append(temp)

products = pd.DataFrame(PRODUCTS)

products["strength"] = products["active_ingredients"].apply(lambda x: " || ".join([item["strength"] if "strength" in item else "nan" for item in x]) if str(x) != "nan" else np.nan)
products["active_ingredients"] = products["active_ingredients"].apply(lambda x: " || ".join([item["name"] for item in x]) if str(x) != "nan" else np.nan)

products["brandName"] = products.brand_name.apply(lambda x: x.lower() if str(x) != "nan" else np.nan)
products["activeIngredients"] = products["active_ingredients"].apply(lambda x: x.lower() if str(x) != "nan" else np.nan)

products["brandName2"] = products.brand_name.apply(lambda x: x.lower().split(" ")[0] if str(x) != "nan" else np.nan)
products["activeIngredients2"] = products["active_ingredients"].apply(lambda x: x.lower().split(" ")[0] if str(x) != "nan" else np.nan)

products2 = products.drop_duplicates(subset=["brandName2"], keep="first")

In [19]:
with open("./ndc.json", "r") as f:
    ndc = json.load(f)
drugs = pd.DataFrame(ndc["results"])
drugs = drugs[["product_ndc", "generic_name", "labeler_name", "brand_name", "active_ingredients", "marketing_category", "dosage_form", 
                 "product_type", "route", "application_number", "brand_name_base", "pharm_class"]]
drugs["pharm_class"] = drugs["pharm_class"].apply(lambda x: " || ".join([item.split(" ")[0] for item in x]) if str(x) != "nan" else np.nan)

drugs["genericName"] = drugs["generic_name"].apply(lambda x: x.lower() if str(x) != "nan" else np.nan)
drugs["brandName"] = drugs["brand_name"].apply(lambda x: x.lower() if str(x) != "nan" else np.nan)
drugs["active_ingredients_new"] = drugs["active_ingredients"].apply(lambda x: " || ".join([item["name"] for item in x]) if str(x) != "nan" else np.nan)
drugs["activeIngredients"] = drugs["active_ingredients_new"].apply(lambda x: x.lower().split(" ")[0] if str(x) != "nan" else np.nan)
drugs["pharmClass"] = drugs["pharm_class"].apply(lambda x: x.lower() if str(x) != "nan" else np.nan)
drugs["brandNameBase"] = drugs["brand_name_base"].apply(lambda x: x.lower() if ((str(x) != "nan") & (str(x) != "None")) else np.nan)

drugs["genericName2"] = drugs["genericName"].apply(lambda x: x.split(" ")[0] if str(x) != "nan" else np.nan)
drugs["brandName2"] = drugs["brandName"].apply(lambda x: x.split(" ")[0] if str(x) != "nan" else np.nan)
drugs["activeIngredients2"] = drugs["activeIngredients"].apply(lambda x: x.split(" ")[0] if str(x) != "nan" else np.nan)
drugs["pharmClass2"] = drugs["pharmClass"].apply(lambda x: x.split(" ")[0] if str(x) != "nan" else np.nan)
drugs["brandNameBase2"] = drugs["brandNameBase"].apply(lambda x: x.split(" ")[0] if str(x) != "nan" else np.nan)

In [20]:
# files = os.listdir("./Labels")
# labels = pd.DataFrame()

# for file in files:
#     print(file)
#     if file.endswith(".json"):
#         with open(f"./Labels/{file}", "r") as f:
#             label = json.load(f)
#         temp = pd.DataFrame(label["results"])
#         temp = temp[["openfda", "description", "indications_and_usage", "contraindications", "warnings", "precautions", "pregnancy", "teratogenic_effects", 
#                             "nursing_mothers", "pediatric_use", "geriatric_use", "adverse_reactions", "do_not_use", "abuse", "dependence"]]
#         for col in temp:
#             temp[col] = temp[col].apply(lambda x: x[0].lower() if ((str(x) != "nan") & (isinstance(x, list))) else x) ### Taking the first elemento nl
#         temp["applnNo"] = temp["openfda"].apply(lambda x: x["application_number"][0] if "application_number" in x else np.nan)
#         temp["product_ndc"] = temp["openfda"].apply(lambda x: " || ".join(x["product_ndc"]) if "application_number" in x else np.nan)
#         temp = temp.drop(["openfda"], axis=1)
#         labels = pd.concat([labels, temp], ignore_index=True)
labels = pd.read_csv("./Labels.csv")

In [21]:
reviews = pd.read_csv("/Users/sid98/OMSA/04_CSE_6242_DVA/ProjectFiles/dva-project/application/data/Review_Dataset.csv", index_col=0)

KeyError: "None of [Index([''], dtype='object')] are in the [columns]"

In [7]:
print("products.brandName2", len(set(reviews.drugName2).intersection(set(products.brandName2))))
print("products.activeIngredients2", len(set(reviews.drugName2).intersection(set(products.activeIngredients2))))
print("drugs.genericName2", len(set(reviews.drugName2).intersection(set(drugs.genericName2))))
print("drugs.brandName2", len(set(reviews.drugName2).intersection(set(drugs.brandName2))))
print("drugs.activeIngredients2", len(set(reviews.drugName2).intersection(set(drugs.activeIngredients2))))
print("drugs.pharmClass2", len(set(reviews.drugName2).intersection(set(drugs.pharmClass2))))
print("drugs.brandNameBase2", len(set(reviews.drugName2).intersection(set(drugs.brandNameBase2))))

products.brandName2 2158
products.activeIngredients2 935
drugs.genericName2 1040
drugs.brandName2 1974
drugs.activeIngredients2 994
drugs.pharmClass2 30
drugs.brandNameBase2 1974


In [7]:
data = reviews.merge(products2, left_on="drugName2", right_on="brandName2", how="left")
df = data[["drugName2", "condition", "review", "rating", "date", "usefulCount", "product_number", "product_number", "reference_drug", "activeIngredients2", "dosage_form", "route", "marketing_status", "applnNo", "strength"]]

In [8]:
applNos = [item for item in df.applnNo.unique() if str(item) != ""]

In [9]:
print("aaplnNo", len(set(df.applnNo).intersection(set(drugs.application_number))))

print("DN : drugs.genericName2", len(set(df.drugName2).intersection(set(drugs.genericName2))))
print("DN : drugs.brandName2", len(set(df.drugName2).intersection(set(drugs.brandName2))))
print("DN : drugs.activeIngredients2", len(set(df.drugName2).intersection(set(drugs.activeIngredients2))))
print("DN : drugs.pharmClass2", len(set(df.drugName2).intersection(set(drugs.pharmClass2))))
print("DN : drugs.brandNameBase2", len(set(df.drugName2).intersection(set(drugs.brandNameBase2))))

print("AI : drugs.genericName2", len(set(df.activeIngredients2).intersection(set(drugs.genericName2))))
print("AI : drugs.brandName2", len(set(df.activeIngredients2).intersection(set(drugs.brandName2))))
print("AI : drugs.activeIngredients2", len(set(df.activeIngredients2).intersection(set(drugs.activeIngredients2))))
print("AI : drugs.pharmClass2", len(set(df.activeIngredients2).intersection(set(drugs.pharmClass2))))
print("AI : drugs.brandNameBase2", len(set(df.activeIngredients2).intersection(set(drugs.brandNameBase2))))

aaplnNo 1239
DN : drugs.genericName2 1040
DN : drugs.brandName2 1974
DN : drugs.activeIngredients2 994
DN : drugs.pharmClass2 30
DN : drugs.brandNameBase2 1974
AI : drugs.genericName2 891
AI : drugs.brandName2 702
AI : drugs.activeIngredients2 888
AI : drugs.pharmClass2 20
AI : drugs.brandNameBase2 702


In [10]:
df2 = df.merge(drugs, left_on="drugName2", right_on="brandNameBase2", how="left")

In [11]:
df2.to_csv("./Merged_Data.csv")

In [12]:
applNos = [item for item in df2.applnNo.unique() if str(item) != ""]
labels2 = labels[labels.applnNo.isin(applNos)].dropna(subset=["applnNo"], axis=0).reset_index(drop=True)
labels2 = labels2.drop_duplicates(subset=["applnNo"], keep="first").reset_index(drop=True)


In [14]:
res = df2.merge(labels2, left_on="applnNo", right_on="applnNo", how="left")

(15642167, 38)

In [25]:
res.to_csv("./FinalDataset.csv")

,drugName2,condition,review,rating,date,usefulCount,product_number,product_number,reference_drug,activeIngredients2_x,dosage_form_x,route_x,marketing_status,applnNo,strength,product_ndc,generic_name,labeler_name,brand_name,active_ingredients,marketing_category,dosage_form_y,product_type,route_y,application_number,brand_name_base,pharm_class,genericName,brandName,active_ingredients_new,activeIngredients,pharmClass,brandNameBase,genericName2,brandName2,activeIngredients2_y,pharmClass2,brandNameBase2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51662-1407,(CHLOROPROCAINE HCI,"HF Acquisition Co LLC, DBA HealthFirst",(CHLOROPROCAINE HCI,"[{'name': 'CHLOROPROCAINE HYDROCHLORIDE', 'str...",NDA,"INJECTION, SOLUTION",HUMAN PRESCRIPTION DRUG,"[INFILTRATION, PERINEURAL]",NDA009435,(CHLOROPROCAINE HCI,Ester || Esters || Local,(chloroprocaine hci,(chloroprocaine hci,CHLOROPROCAINE HYDROCHLORIDE,chloroprocaine,ester || esters || local,(chloroprocaine hci,(chloroprocaine,(chloroprocaine,chloroprocaine,ester,(chloroprocaine
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75936-603,ZINC OXIDE,"Supergoop, LLC",(Glow)Setting 100% Mineral Powder SPF 35,"[{'name': 'ZINC OXIDE', 'strength': '24.9 g/10...",OTC MONOGRAPH FINAL,POWDER,HUMAN OTC DRUG,[TOPICAL],M020,(Glow)Setting 100% Mineral Powder SPF 35,NaN,zinc oxide,(glow)setting 100% mineral powder spf 35,ZINC OXIDE,zinc,NaN,(glow)setting 100% mineral powder spf 35,zinc,(glow)setting,zinc,NaN,(glow)setting
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75936-166,Zinc Oxide,"Supergoop, LLC",(Re) Setting 100% mineral Powder Light Broad S...,"[{'name': 'ZINC OXIDE', 'strength': '24.7 g/10...",OTC MONOGRAPH FINAL,POWDER,HUMAN OTC DRUG,[TOPICAL],part352,(Re) Setting 100% mineral Powder Light Broad S...,NaN,zinc oxide,(re) setting 100% mineral powder light broad s...,ZINC OXIDE,zinc,NaN,(re) setting 100% mineral powder light broad s...,zinc,(re),zinc,NaN,(re)
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75936-168,Zinc Oxide,"Supergoop, LLC",(Re) Setting 100% mineral Powder Deep Broad Sp...,"[{'name': 'ZINC OXIDE', 'strength': '24.7 g/10...",OTC MONOGRAPH FINAL,POWDER,HUMAN OTC DRUG,[TOPICAL],part352,(Re) Setting 100% mineral Powder Deep Broad Sp...,NaN,zinc oxide,(re) setting 100% mineral powder deep broad sp...,ZINC OXIDE,zinc,NaN,(re) setting 100% mineral powder deep broad sp...,zinc,(re),zinc,NaN,(re)
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75936-167,Zinc Oxide,"Supergoop, LLC",(RE) Setting 100% Mineral Powder Medium Broad ...,"[{'name': 'ZINC OXIDE', 'strength': '24.7 g/10...",OTC MONOGRAPH FINAL,POWDER,HUMAN OTC DRUG,[TOPICAL],part352,(RE) Setting 100% Mineral Powder Medium Broad ...,NaN,zinc oxide,(re) setting 100% mineral powder medium broad ...,ZINC OXIDE,zinc,NaN,(re) setting 100% mineral powder medium broad ...,zinc,(re),zinc,NaN,(re)


In [15]:
df2 = df2[["drugName2", "condition", "review", "rating", "date", "usefulCount", ]

,drugName2,condition,review,rating,date,usefulCount,product_number,product_number,reference_drug,activeIngredients2_x,dosage_form_x,route_x,marketing_status,applnNo,strength,product_ndc,generic_name,labeler_name,brand_name,active_ingredients,marketing_category,dosage_form_y,product_type,route_y,application_number,brand_name_base,pharm_class,genericName,brandName,active_ingredients_new,activeIngredients,pharmClass,brandNameBase,genericName2,brandName2,activeIngredients2_y,pharmClass2,brandNameBase2
0,mirtazapine,Depression,"""I've tried a few antidepressants over the yea...",10.0,"February 28, 2012",22,003,003,No,mirtazapine,TABLET,ORAL,Discontinued,ANDA076241,45MG,76483-112,Mirtazapine,SQUARE PHARMACEUTICALS LIMITED,Mirtazapine,"[{'name': 'MIRTAZAPINE', 'strength': '45 mg/1'}]",ANDA,"TABLET, ORALLY DISINTEGRATING",HUMAN PRESCRIPTION DRUG,[ORAL],ANDA205798,Mirtazapine,NaN,mirtazapine,mirtazapine,MIRTAZAPINE,mirtazapine,NaN,mirtazapine,mirtazapine,mirtazapine,mirtazapine,NaN,mirtazapine
1,mirtazapine,Depression,"""I've tried a few antidepressants over the yea...",10.0,"February 28, 2012",22,003,003,No,mirtazapine,TABLET,ORAL,Discontinued,ANDA076241,45MG,45865-382,Mirtazapine,Medsource Pharmaceuticals,Mirtazapine,"[{'name': 'MIRTAZAPINE', 'strength': '15 mg/1'}]",ANDA,"TABLET, FILM COATED",HUMAN PRESCRIPTION DRUG,[ORAL],ANDA076921,Mirtazapine,NaN,mirtazapine,mirtazapine,MIRTAZAPINE,mirtazapine,NaN,mirtazapine,mirtazapine,mirtazapine,mirtazapine,NaN,mirtazapine
2,mirtazapine,Depression,"""I've tried a few antidepressants over the yea...",10.0,"February 28, 2012",22,003,003,No,mirtazapine,TABLET,ORAL,Discontinued,ANDA076241,45MG,50090-2800,Mirtazapine,A-S Medication Solutions,Mirtazapine,"[{'name': 'MIRTAZAPINE', 'strength': '15 mg/1'}]",ANDA,"TABLET, FILM COATED",HUMAN PRESCRIPTION DRUG,[ORAL],ANDA076921,Mirtazapine,NaN,mirtazapine,mirtazapine,MIRTAZAPINE,mirtazapine,NaN,mirtazapine,mirtazapine,mirtazapine,mirtazapine,NaN,mirtazapine
3,mirtazapine,Depression,"""I've tried a few antidepressants over the yea...",10.0,"February 28, 2012",22,003,003,No,mirtazapine,TABLET,ORAL,Discontinued,ANDA076241,45MG,63187-206,Mirtazapine,Proficient Rx LP,Mirtazapine,"[{'name': 'MIRTAZAPINE', 'strength': '15 mg/1'}]",ANDA,"TABLET, FILM COATED",HUMAN PRESCRIPTION DRUG,[ORAL],ANDA077666,Mirtazapine,NaN,mirtazapine,mirtazapine,MIRTAZAPINE,mirtazapine,NaN,mirtazapine,mirtazapine,mirtazapine,mirtazapine,NaN,mirtazapine
4,mirtazapine,Depression,"""I've tried a few antidepressants over the yea...",10.0,"February 28, 2012",22,003,003,No,mirtazapine,TABLET,ORAL,Discontinued,ANDA076241,45MG,68788-7492,Mirtazapine,"Preferred Pharmaceuticals, Inc.",Mirtazapine,"[{'name': 'MIRTAZAPINE', 'strength': '15 mg/1'}]",ANDA,"TABLET, FILM COATED",HUMAN PRESCRIPTION DRUG,[ORAL],ANDA076921,Mirtazapine,NaN,mirtazapine,mirtazapine,MIRTAZAPINE,mirtazapine,NaN,mirtazapine,mirtazapine,mirtazapine,mirtazapine,NaN,mirtazapine
